In [0]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp, lit, trim
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from datetime import datetime

# Inicializa a sessão
spark = SparkSession.builder.getOrCreate()

In [0]:
catalog       = "workspace"
schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
# Padronização Silver
# Limpeza, tipos e nomes
df_bronze = spark.table(f"{catalog}.{schema_bronze}.pessoas")

df_silver = (df_bronze
             .select(
                   (df_bronze.Codigo).cast("int").alias("codigo")
                 , (F.trim(df_bronze.PrimeiroNome)).alias("primeiro_nome")
                 , (F.trim(df_bronze.UltimoNome).alias("ultimo_nome"))
                 , (F.lower(F.trim(df_bronze.email))).alias("email")
                 , (F.trim(df_bronze.Genero)).alias("genero")
                 , (F.trim(df_bronze.Avatar)).alias("avatar")
                 , (F.trim(df_bronze.Pais)).alias("pais")
                 , F.to_date(df_bronze.Nascimento, "dd/MM/yyyy").alias("data_nascimento")
                 , (df_bronze._ingestion_ts)
                 )
             .dropDuplicates(["codigo"])
             .withColumn("_silver_ts", F.current_timestamp())
             .withColumn("nome_completo", F.concat_ws(" ", F.col("primeiro_nome"), F.col("ultimo_nome")))
            )

In [0]:
(df_silver
 .write
 .mode("overwrite")
 .format("delta")
 .option("overwriteSchema", "true")
 .saveAsTable(f"{catalog}.{schema_silver}.pessoas")
)

print("Tabela Silver de pessoas salva com sucesso! ✨")